In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from keras.models import sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

In [3]:
num_classes=5
img_rows,img_cols=48,48 #size of image 
batch_size=8 # how many images want to give at once 

In [4]:
train_data=r'D:/Facial_expression/train' #24282 images 
validation_data=r'D:/Facial_expression/validation' #5937

In [5]:
#ImageDataGenerator class allows you to randomly rotate images through any degree between 0 and 360 by providing an integer value in the rotation_range argument.
train_data_generator=ImageDataGenerator(rotation_range=30,width_shift_range=0.4,
                                        height_shift_range=0.4,shear_range=30,
                                        zoom_range=0.3,horizontal_flip=True,
                                        vertical_flip=True,rescale=1./255) 
# as image data is very big we have to scale the data
validation_data_generator=ImageDataGenerator(rescale=1./255)
train_generator=train_data_generator.flow_from_directory(train_data,color_mode='grayscale',
                                                         target_size=(img_rows,img_cols),
                                                         batch_size=batch_size,
                                                         class_mode='categorical',shuffle=True)
validation_generator=validation_data_generator.flow_from_directory(validation_data,
                                                                   color_mode='grayscale',
                                                                   target_size=(img_rows,img_cols),
                                                                   batch_size=batch_size,
                                                                   class_mode='categorical',
                                                                   shuffle=True)


Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [6]:
model=keras.Sequential()

In [7]:
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [8]:
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [9]:
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [10]:
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [11]:
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


In [12]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [13]:
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [14]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                        

In [20]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [21]:
checkpoint = ModelCheckpoint(r'D:/Facial_expression/Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

In [22]:
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

In [23]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

In [24]:
callbacks = [earlystop,checkpoint,reduce_lr]

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [27]:
nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

In [29]:
history=model.fit(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
3022/3022 [==============================] - ETA: 0s - loss: 1.6578 - accuracy: 0.2605
Epoch 1: val_loss improved from inf to 1.53890, saving model to D:/Facial_expression\Emotion_little_vgg.h5
3022/3022 [==============================] - 497s 164ms/step - loss: 1.6578 - accuracy: 0.2605 - val_loss: 1.5389 - val_accuracy: 0.3010 - lr: 0.0010
Epoch 2/25
3022/3022 [==============================] - ETA: 0s - loss: 1.5670 - accuracy: 0.2842
Epoch 2: val_loss did not improve from 1.53890
3022/3022 [==============================] - 411s 136ms/step - loss: 1.5670 - accuracy: 0.2842 - val_loss: 1.5457 - val_accuracy: 0.3067 - lr: 0.0010
Epoch 3/25
3022/3022 [==============================] - ETA: 0s - loss: 1.5607 - accuracy: 0.2904
Epoch 3: val_loss did not improve from 1.53890
3022/3022 [==============================] - 376s 124ms/step - loss: 1.5607 - accuracy: 0.2904 - val_loss: 1.5716 - val_accuracy: 0.3027 - lr: 0.0010
Epoch 4/25
3022/3022 [==============================] -